#Toxic/hate speech classification


In [22]:
# !pip install -U -q transformers
!pip install transformers 
!pip install sentencepiece

In [23]:
import os

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from transformers import TFAutoModel, AutoTokenizer
from tqdm.notebook import tqdm
from tokenizers import Tokenizer, models, pre_tokenizers, decoders, processors
from transformers import AutoTokenizer, AutoConfig


#TPU Configs

In [24]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.92.44.138:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.92.44.138:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.92.44.138:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


#Functions

In [25]:
# Configuration
EPOCHS = 3
BATCH_SIZE = 64 
MAX_LEN = 200
PATIENCE = 1
LEARNING_RATE = 1e-5

In [26]:
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification
from transformers import (XLMRobertaConfig, XLMRobertaTokenizer, TFXLMRobertaModel)            
from transformers import AutoTokenizer, AutoConfig, AutoModel    

PRETRAINED_MODEL_TYPES = {
    'xlmroberta': (AutoConfig,  AutoModel, AutoTokenizer, 'akhooli/xlm-r-large-arabic-toxic')
}

# model_class,model = AutoModelForSequenceClassification.from_pretrained("akhooli/xlm-r-large-arabic-toxic")

config_class, model_class, tokenizer_class, model_name = PRETRAINED_MODEL_TYPES['xlmroberta']

# Download vocabulary from huggingface.co and cache.
# tokenizer = tokenizer_class.from_pretrained(model_name) 
# from_pt=True,
tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=False) #fast tokenizer

tokenizer

PreTrainedTokenizer(name_or_path='akhooli/xlm-r-large-arabic-toxic', vocab_size=250002, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'})

In [27]:
PRE_TRAINED_MODEL = 'Hate-speech-CNERG/dehatebert-mono-arabic'
# PRE_TRAINED_MODEL = 'bert-base-multilingual-cased'
# PRE_TRAINED_MODEL = 'xlm-roberta-large'
tokenizer = transformers.AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL)
from transformers import AutoModelForSequenceClassification

model = AutoModel.from_pretrained(PRE_TRAINED_MODEL,
                                                      num_labels = 3,
                                                      output_attentions = False,
                                                      output_hidden_states = False)

In [28]:
def build_model(transformer, max_len=512):
    """
    function for training the model
    """
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(1, activation='sigmoid')(cls_token)
    
    model = Model(inputs=input_word_ids, outputs=out)
    model.compile(Adam(lr=3e-5), loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()])
    # changed from 1e-5 to 3e-5
    return model

In [29]:
with strategy.scope():
    transformer_layer = (
        transformers
        .AutoModelForSequenceClassification.from_pretrained("Hate-speech-CNERG/dehatebert-mono-arabic")
        # .TFAutoModel.from_pretrained('jplu/tf-xlm-roberta-large')
    )
    model = build_model(transformer_layer, max_len=MAX_LEN)

AttributeError: ignored

#Load text data

In [30]:
train = pd.read_excel("/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/train.xlsx")
# n_train_steps = train.shape[0] 
valid = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/dev.xlsx')
# n_valid_steps = valid.shape[0] 

test = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/test.xlsx')
sub = pd.read_excel('/content/drive/MyDrive/MasterThesis/DataSet-forUsing/semeval2020/sample_submission.xlsx')
# gc.collect()


In [31]:
def encode(df, tokenizer, max_len=512):
    
    pairs = df['tweets'].values.astype(str).tolist() #shape=[num_examples]
    # pairs = df['tweets'].values.astype(str) #shape=[num_examples]
    
    print ("Encoding...")
    encoded_dict = tokenizer.batch_encode_plus(pairs, max_length=max_len, padding=True, truncation=True, 
                                               add_special_tokens=True, return_attention_mask=True)
    print ("Complete")
    
    input_word_ids = tf.convert_to_tensor(encoded_dict['input_ids'], dtype=tf.int32) #shape=[num_examples, max_len])
    input_mask = tf.convert_to_tensor(encoded_dict['attention_mask'], dtype=tf.int32) #shape=[num_examples, max_len]
    
    inputs = {
        'input_word_ids': input_word_ids,
        'input_mask': input_mask}    
    
    return inputs

In [32]:
train_input = encode(train, tokenizer=tokenizer, max_len=MAX_LEN)
vaild_input = encode(valid, tokenizer=tokenizer, max_len=MAX_LEN)
test_input = encode(test, tokenizer=tokenizer, max_len=MAX_LEN)

Encoding...
Complete
Encoding...
Complete
Encoding...
Complete


In [33]:
# y_train = train.hs.values.tolist()
# y_valid = valid.hs.values

In [34]:
# # instantiating the model in the strategy scope creates the model on the TPU
# with strategy.scope():
#     # model = build_model(MAX_LEN)
#     # transformer_layer = TFAutoModel.from_pretrained(model_name)
#     model = build_model(max_len=MAX_LEN)
#     model.summary()

#Remove

In [35]:
# from transformers import BertTokenizer

# # Load the BERT tokenizer.
# print('Loading BERT tokenizer...')
# # tokenizer = AutoTokenizer.from_pretrained("akhooli/xlm-r-large-arabic-toxic",use_fast=False)


In [36]:
# tokenizer

In [37]:
# def build_model(max_len=512):
    
#     # tf.random.set_seed(12345) # For reproducibility
    
#     # The bare XLM-RoBERTa Model transformer outputting raw hidden-states without any specific head on top.
#     encoder = model_class.from_pretrained(model_name)
# #     encoder = TFAutoModel.from_pretrained(model_name)
#     # tokenized_text = tokenizer.prepare_seq2seq_batch([text], return_tensors='pt')


#     input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
#     input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    
#     # Extract pretrained embedding vectors
#     embedding = encoder([input_word_ids, input_mask])[0] # shape=(batch_size, max_len, embed_size)
#     # embedding = encoder(input_word_ids)[0] # shape=(batch_size, max_len, embed_size)
#     # We pass the embedding vectors of only the 'cls' token (at index=0) to the dense layer
#     sequence_output = embedding[:,0,:] #shape=(batch_size, embed_size)
   
#     # Add a classification layer
#     # output = tf.keras.layers.Dense(1, activation="sigmoid")(sequence_output)  
#     output = tf.keras.layers.Dense(2, activation="softmax")(sequence_output)  

#     model = tf.keras.Model(inputs=[input_word_ids, input_mask], outputs=output)
#     # model = tf.keras.Model(inputs=input_word_ids, outputs=output)
#     model.compile(tf.keras.optimizers.Adam(lr=LEARNING_RATE), loss='binary_crossentropy',  metrics=[tf.keras.metrics.AUC(name='auc'), 'accuracy'])
    
#     return model

In [38]:
# # Print the original sentence.
# print(' Original: ', x_train[0])

# # Print the sentence split into tokens.
# print('Tokenized: ', tokenizer.tokenize(x_train[0]))

# # Print the sentence mapped to token ids.
# print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x_train[0])))

In [39]:
# def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
#     """
#     Encoder for encoding the text into sequence of integers for BERT Input
#     """
#     # tokenizer.enable_truncation(max_length=maxlen)
#     # tokenizer.enable_padding(max_length=maxlen)
#     tokenizer.enable_truncation(max_length=maxlen)
#     if enable_padding:
#        tokenizer.enable_padding(max_length=maxlen)
#     all_ids = []
    
#     for i in tqdm(range(0, len(texts), chunk_size)):
#         text_chunk = texts[i:i+chunk_size].tolist()
#         encs = tokenizer.encode_batch(text_chunk)
#         all_ids.extend([enc.ids for enc in encs])
    
#     return np.array(all_ids)

In [40]:
# def combine_qa_ids(q_ids, a_ids, tokenizer, maxlen=512):
#     """
#     Given two arrays of IDs (questions and answers) created by
#     `fast_encode`, we combine and pad them.
#     Inputs:
#         tokenizer: The original tokenizer (not the fast_tokenizer)
#     """
#     combined_ids = []

#     for i in tqdm(range(q_ids.shape[0])):
#         ids = []
#         ids.append(tokenizer.cls_token_id)
#         ids.extend(q_ids[i])
#         ids.append(tokenizer.sep_token_id)
#         ids.extend(a_ids[i])
#         ids.append(tokenizer.sep_token_id)
#         ids.extend([tokenizer.pad_token_id] * (maxlen - len(ids)))

#         combined_ids.append(ids)
    
#     return np.array(combined_ids)

In [41]:
# def encode_qa(questions, answers, tokenizer, chunk_size=256, maxlen=512):
#     """
#     https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
#     """
#     tokenizer.enable_truncation(max_length=maxlen)
#     tokenizer.enable_padding(max_length=maxlen)
#     all_ids = []
    
#     for i in tqdm(range(0, len(questions), chunk_size)):
#         q_chunk = questions[i:i+chunk_size].tolist()
#         a_chunk = answers[i:i+chunk_size].tolist()
#         text_chunk = list(zip(q_chunk, a_chunk))
        
#         encs = tokenizer.encode_batch(text_chunk)
#         all_ids.extend([enc.ids for enc in encs])
    
#     return np.array(all_ids)

In [42]:
# def truncate_text(text, tokenizer, chunk_size=256, maxlen=256):
#     """
#     Ensure that the text does not have more than maxlen tokens
#     """
#     tokenizer.enable_truncation(max_length=maxlen)
#     all_norm_str = []
    
#     for i in tqdm(range(0, len(text), chunk_size)):
#         chunk = text[i:i+chunk_size].tolist()
#         encs = tokenizer.encode_batch(chunk)
#         all_norm_str.extend([str(enc.normalized_str) for enc in encs])
    
#     return all_norm_str

In [43]:
# #IMP DATA FOR CONFIG

# AUTO = tf.data.experimental.AUTOTUNE


# # Configuration
# EPOCHS = 3
# BATCH_SIZE = 16 * strategy.num_replicas_in_sync
# MAX_LEN = 192

In [44]:
# from tokenizers import BertWordPieceTokenizer

# # First load the real tokenizer
# tokenizer = AutoTokenizer.from_pretrained('akhooli/xlm-r-large-arabic-toxic')
# # Save the loaded tokenizer locally
# # tokenizer.save_pretrained('.')
# # Reload it with the huggingface tokenizers library
# # fast_tokenizer = BertWordPieceTokenizer('/content/saved_model/vocab.txt', do_lower_case=False)
# # fast_tokenizer=BertTokenizer('/content/saved_model/vocab.txt', do_lower_case=False)
# # fast_tokenizer=BertTokenizer.from_pretrained('./content/saved_model/vocab.txt')
# # fast_tokenizer
# # Save the loaded tokenizer locally
# # tokenizer.save_pretrained('.')
# # Reload it with the huggingface tokenizers library
# from transformers import BertTokenizer
# sequence = "Hello, y'all! How are you Tokenizer 😁 ?"

# tokenizerBT = BertTokenizer('/content/saved_model/vocab.txt')
# tokenized_sequenceBT = tokenizerBT.encode(sequence)
# print(tokenized_sequenceBT)
# # fast_tokenizer = BertWordPieceTokenizer('/content/saved_model/vocab.txt', lowercase=True, add_special_tokens=False)
# # fast_tokenizer

In [45]:
# tokenizerBT

In [46]:
# x_train = fast_encode(train.tweets.astype(str), tokenizer, maxlen=MAX_LEN)
# x_valid = fast_encode(valid.tweets.astype(str), tokenizer, maxlen=MAX_LEN)
# x_test = fast_encode(test.tweets.astype(str), tokenizer, maxlen=MAX_LEN)

# y_train = train.hs.values
# y_valid = valid.hs.values

In [47]:
# train_dataset = (
#     # tf.convert_to_tensor((x_train, y_train))
#     tf.data.Dataset.from_tensor_slices((x_train, y_train))
#     .repeat()
#     .shuffle(2048)
#     .batch(BATCH_SIZE)
#     .prefetch(AUTO)
# )

# valid_dataset = (
#     tf.data.Dataset
#     .from_tensor_slices((x_valid, y_valid))
#     .batch(BATCH_SIZE)
#     .cache()
#     .prefetch(AUTO)
# )

# test_dataset = (
#     tf.data.Dataset
#     .from_tensor_slices(x_test)
#     .batch(BATCH_SIZE)
# )


In [48]:
# # Tokenize all of the sentences and map the tokens to thier word IDs.
# input_ids = []

# # For every sentence...
# for tweet in tweets:
#     # `encode` will:
#     #   (1) Tokenize the sentence.
#     #   (2) Prepend the `[CLS]` token to the start.
#     #   (3) Append the `[SEP]` token to the end.
#     #   (4) Map tokens to their IDs.
#     encoded_tweets = tokenizer.encode(
#                         str(tweet),                      # Sentence to encode.
#                         add_special_tokens = True, # Add '[CLS]' and '[SEP]'

#                         # This function also supports truncation and conversion
#                         # to pytorch tensors, but we need to do padding, so we
#                         # can't use these features :( .
#                         #max_length = 128,          # Truncate all sentences.
#                         #return_tensors = 'pt',     # Return pytorch tensors.
#                    )
    
#     # Add the encoded sentence to the list.
#     input_ids.append(encoded_tweets)

# # Print tweet 0, now as a list of IDs.
# print('Original: ', x_train[0])
# print('Token IDs:', input_ids[0])

In [50]:
# print('Max sentence length: ', max([len(sen) for sen in input_ids]))

In [51]:
# from keras.preprocessing.sequence import pad_sequences

# # Set the maximum sequence length.
# # I've chosen 64 somewhat arbitrarily. It's slightly larger than the
# # maximum training sentence length of 47...
# MAX_LEN = 192

# print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)

# print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))

# # Pad our input tokens with value 0.
# # "post" indicates that we want to pad and truncate at the end of the sequence,
# # as opposed to the beginning.
# input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", 
#                           value=0, truncating="post", padding="post")

# print('\nDone.')

In [52]:
# # Create attention masks
# attention_masks = []

# # For each sentence...
# for tweet in input_ids:
    
#     # Create the attention mask.
#     #   - If a token ID is 0, then it's padding, set the mask to 0.
#     #   - If a token ID is > 0, then it's a real token, set the mask to 1.
#     att_mask = [int(token_id > 0) for token_id in tweet]
    
#     # Store the attention mask for this sentence.
#     attention_masks.append(att_mask)

In [53]:
# # Use train_test_split to split our data into train and validation sets for
# # training
# from sklearn.model_selection import train_test_split

# # Use 90% for training and 10% for validation.
# train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, 
#                                                             random_state=2018, test_size=0.1)
# # Do the same for the masks.
# train_masks, validation_masks, _, _ = train_test_split(attention_masks, labels,
#                                              random_state=2018, test_size=0.1)

In [54]:
# import torch

In [55]:
# # # Convert all inputs and labels into torch tensors, the required datatype 
# # for our model.
# train_inputs = torch.tensor(x_train.shape[0])
# validation_inputs = torch.tensor(valid.tweets)

# train_labels = torch.tensor(train_labels)
# validation_labels = torch.tensor(validation_labels)

# train_masks = torch.tensor(train_masks)
# validation_masks = torch.tensor(validation_masks)


# train_seq = torch.tensor(tokens_train['input_ids'])
# train_mask = torch.tensor(tokens_train['attention_mask'])
# train_y = torch.tensor(train_labels.tolist())

In [56]:
# def build_model(transformer, max_len=512):
#     """
#     https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
#     """
#     input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
#     attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
#     sequence_output = transformer({"input_ids": input_word_ids, "attention_mask": attention_mask})[0]
#     cls_token = sequence_output[:, 0, :]
#     out = Dense(1, activation='sigmoid')(cls_token)
    
#     model = Model(inputs={
#         "input_ids": input_word_ids,
#         "attention_mask": attention_mask
#     }, outputs=out)
#     model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
#     return model

In [57]:
# # Detect hardware, return appropriate distribution strategy
# try:
#     # TPU detection. No parameters necessary if TPU_NAME environment variable is
#     # set: this is always the case on Kaggle.
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     tpu = None

# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
#     strategy = tf.distribute.get_strategy()

# print("REPLICAS: ", strategy.num_replicas_in_sync)

In [58]:
# MAX_LEN = 192
# MODEL = 'akhooli/xlm-r-large-arabic-toxic'
# AUTO = tf.data.experimental.AUTOTUNE
# BATCH_SIZE = 4 * strategy.num_replicas_in_sync
# tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [59]:
# x_train = dict_encode(train.tweets.values, tokenizer, maxlen=MAX_LEN)
# y_train = train.hs.values

# del train
# gc.collect()

# x_valid = dict_encode(valid.tweets.values, tokenizer, maxlen=MAX_LEN)
# y_valid = valid.hs.values
# del valid
# gc.collect()
# x_test = dict_encode(test.tweets.values, tokenizer, maxlen=MAX_LEN)

In [60]:
# First load the real tokenizer
# tokenizer = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
# tokenizer = AutoTokenizer.from_pretrained("akhooli/xlm-r-large-arabic-toxic",use_fast=False)

# # Save the loaded tokenizer locally
# tokenizer.save_pretrained('./models/tokenizer/')
# Reload it with the huggingface tokenizers library
# fast_tokenizer = DistilBertTokenizer.from_pretrained('/content/models/tokenizer')
# fast_tokenizer = BertWordPieceTokenizer('vocab.txt')
# fast_tokenizer

# BASE_MODEL = "distilbert-base-multilingual-cased"
# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
# tokenizer.save_pretrained("./models/tokenizer/")


##Create fast tokenizer

In [61]:
# AUTO = tf.data.experimental.AUTOTUNE

# # Data access
# # GCS_DS_PATH = KaggleDatasets().get_gcs_path()

# # Configuration
# EPOCHS = 2
# BATCH_SIZE = 16 * strategy.num_replicas_in_sync
# MAX_LEN = 192
# MODEL = 'akhooli/xlm-r-large-arabic-toxic'

In [62]:
# def build_model(transformer, max_len=512):
#     """
#     https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
#     """
#     input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
#     sequence_output = transformer(input_word_ids)[0]
#     cls_token = sequence_output[:, 0, :]
#     out = Dense(1, activation='sigmoid')(cls_token)
    
#     model = Model(inputs=input_word_ids, outputs=out)
#     model.compile(Adam(lr=1e-5), loss='binary_crossentropy', metrics=['accuracy'])
    
#     return model

In [63]:
# # def regular_encode(texts, tokenizer, maxlen=512):
# #     enc_di = tokenizer.batch_encode_plus(
# #         texts,
# #         return_tensors='tf',
# #         return_attention_masks=False, 
# #         return_token_type_ids=True,
# #         pad_to_max_length=True,
# #         max_length=10
# #         # texts, 
# #         # return_attention_masks=True, 
# #         # return_token_type_ids=False,
# #         # pad_to_max_length=True,
# #         # padding = 'max_length',
# #         # max_length=maxlen,
# #         # return_tensors='pt'

# #         #  add_special_tokens=False,
# #         #                                       # max_length=10,
# #         #                                       # pad_to_max_length=True,
# #         #                                       return_tensors='pt',
# #         #                                       return_attention_mask=True,
# #         #                                       return_overflowing_tokens=False
# #     )
    
# #     return np.array(enc_di['input_ids'])

# def regular_encode(texts, tokenizer, maxlen=512):
#     enc_di = tokenizer.batch_encode_plus(
#         texts, 
#         # return_attention_masks=False, 
#         # return_token_type_ids=False,
#         pad_to_max_length=True,
#         max_length=maxlen
#     )
#     # batch_encode_plus(inp_raw, max_length=20, pad_to_max_length=True)
    
#     return np.array(enc_di['input_ids'])

In [64]:
# def fast_encode(texts, tokenizer, chunk_size=256, maxlen=512):
#     """
#     https://www.kaggle.com/xhlulu/jigsaw-tpu-distilbert-with-huggingface-and-keras
#     """
#     tokenizer.enable_truncation(max_length=maxlen)
#     tokenizer.enable_padding(max_length=maxlen)
#     all_ids = []
    
#     for i in tqdm(range(0, len(texts), chunk_size)):
#         text_chunk = texts[i:i+chunk_size].tolist()
#         encs = tokenizer.encode_batch(text_chunk)
#         all_ids.extend([enc.ids for enc in encs])
    
#     return np.array(all_ids)

In [65]:
# !pip install sentencepiece

In [66]:
# First load the real tokenizer
# tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [67]:
# from tokenizers import BertWordPieceTokenizer

# # # First load the real tokenizer
# # # tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

# # Save the loaded tokenizer locally

# save_path = "akhooli/xlm-r-large-arabic-toxic/"
# if not os.path.exists(save_path):
#     os.makedirs(save_path)
# tokenizer.save_pretrained(save_path)

# # Load the fast tokenizer from saved file
# tokenizer = BertWordPieceTokenizer("akhooli/xlm-r-large-arabic-toxic/vocab.txt", lowercase=True)



# ,from_pt=True
# tokenizer = AutoTokenizer.from_pretrained('akhooli/xlm-r-large-arabic-toxic',use_fast=False)
# tokenizer = AutoTokenizer.from_pretrained('Hate-speech-CNERG/dehatebert-mono-arabic')
# tokenizer.save_pretrained('TEST/tokenizer')

# tokenizer = AutoTokenizer.from_pretrained('TEST/tokenizer', config=AutoConfig.from_pretrained("akhooli/xlm-r-large-arabic-toxic"))

In [68]:
# len(tokenizer.vocab)


In [69]:

# bert_keys = []

# for token in tokenizer.vocab.keys():
    
#     bert_keys.append(token)
    
    
# print(bert_keys[101])
# print(bert_keys[102])
# print(bert_keys[0])

# print(bert_keys[7592])
# print(bert_keys[2088])

In [70]:
# # x_train=tokenizer.encode_batch(train.tweets.values.tolist())
# x_train = fast_encode(train.tweets.astype(str), tokenizer, maxlen=MAX_LEN)
# x_valid = regular_encode(valid.tweets.values, tokenizer, maxlen=MAX_LEN)
# x_test = regular_encode(test.tweets.values, tokenizer, maxlen=MAX_LEN)

# y_train = train.hs.values
# y_valid = valid.hs.values

In [71]:
# bert_vocab = tokenizer.vocab
# bert_vocab
# print(bert_vocab['[CLS]'])
# print(bert_vocab['[SEP]'])
# print(bert_vocab['[PAD]'])

# print(bert_vocab['hello'])
# print(bert_vocab['world'])

In [72]:
# # Get the lists of sentences and their labels.
# x_train = train.tweets.values
# y_train = train.hs.values
# x_vaild = valid.tweets.values
# y_valid = valid.hs.values
# x_test = train.tweets.values
# # y_train = train.hs.values